In [17]:
import os
import sys
import shutil
import subprocess
sys.path.append("../")

from pathlib import Path

In [18]:
from HandsFreeDocking.tools.Protein_Preparation import ProteinPreparation_Chimera

In [19]:
resource_dir = Path("../examples/")
protein = resource_dir / "LAG3_Moloc_1.pdb"
crystal = resource_dir / "Fake_Crystal.sdf"
to_dock = resource_dir / "Fake_Crystal.sdf"

protein_abs = protein.resolve()
crystal_abs = crystal.resolve()
to_dock_abs = to_dock.resolve()

assert protein_abs.exists()
assert crystal_abs.exists()
assert to_dock_abs.exists()

In [20]:
env = os.environ.copy()
env["RBT_ROOT"] = "/home/hitesit/Software/rxdock/rxdock_installation"
env["PATH"] = f"{env['RBT_ROOT']}/bin:{env.get('PATH', '')}"
env["LD_LIBRARY_PATH"] = f"{env['RBT_ROOT']}/lib/x86_64-linux-gnu:{env.get('LD_LIBRARY_PATH', '')}"
env["RBT_HOME"] = os.getcwd()

In [21]:
# Run a command with subprocess
subprocess.run("rbdock", env=env)

***********************************************
The RxDock molecular docking program is licensed under GNU LGPL version 3.
RxDock is maintained by Vedran Miletić, Patrik Nikolić, and Luka Vretenar.
Visit https://www.rxdock.org/ for more information.
Executable:	rbdock/0.1.0
Library:	librxdock.so/0.1.0
RBT_ROOT:	/home/hitesit/Software/rxdock/rxdock_installation
RBT_HOME:	/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/Prototypes
Current dir:	/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/Prototypes
Date:		Fri May 16 17:56:27 2025
***********************************************


CompletedProcess(args='rbdock', returncode=1)

Missing required parameter(s): -i, -r, or -p


In [22]:
prep = ProteinPreparation_Chimera()
protein_mols = prep(protein_abs, Path("Protein.mol2"))

Preparing protein with Chimera ...


In [23]:
protein_mols

PosixPath('Protein.mol2')

In [24]:
prm = f"""RBT_PARAMETER_FILE_V1.00
TITLE gart_DUD
RECEPTOR_FILE {str(protein_mols)}
RECEPTOR_FLEX 3.0
##############################################
## CAVITY DEFINITION: REFERENCE LIGAND METHOD
##############################################
SECTION MAPPER
    SITE_MAPPER RbtLigandSiteMapper
    REF_MOL {str(crystal_abs)}
    RADIUS 6.0
    SMALL_SPHERE 1.0
    MIN_VOLUME 100
    MAX_CAVITIES 1
    VOL_INCR 0.0
    GRIDSTEP 0.5
END_SECTION
############################
## CAVITY RESTRAINT PENALTY
SECTION CAVITY
    SCORING_FUNCTION RbtCavityGridSF
    WEIGHT 1.0
END_SECTION
"""

prmtop_file = "probe.prmtop"
with open(prmtop_file, "w") as f:
    f.write(prm)

In [25]:
params = [
    "rbcavity",
    "-W",
    "-d",
    "-r",
    "probe.prmtop"
]

result = subprocess.run(params, capture_output=True, text=True, env=env)
print(result.stdout)
if result.returncode != 0:
    print(f"Error: {result.stderr}")

***********************************************
The RxDock molecular docking program is licensed under GNU LGPL version 3.
RxDock is maintained by Vedran Miletić, Patrik Nikolić, and Luka Vretenar.
Visit https://www.rxdock.org/ for more information.
Executable:	rbcavity/0.1.0
Library:	librxdock.so/0.1.0
RBT_ROOT:	/home/hitesit/Software/rxdock/rxdock_installation
RBT_HOME:	/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/Prototypes
Current dir:	/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/Prototypes
Date:		Fri May 16 17:56:29 2025
***********************************************
Command line arguments:
-r probe.prmtop
-W
-d

MAPPER
CLASS	RbtLigandSiteMapper
ENABLED	TRUE
GRIDSTEP	0.5
MAX_CAVITIES	1
MIN_VOLUME	100
NAME	MAPPER
RADIUS	6.0
REF_MOL	/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/examples/Fake_Crystal.sdf
SMALL_SPHERE	1.0
TRACE	0
VOL_INCR	0.0


DOCKING SITE
Total volume 1306.75 A^3
Cavity #1	Size=10454 points; Vol=1306.75 A^3

In [28]:
params = [
    "rbdock",
    "-i", str(crystal_abs),
    "-o", "docking_results",
    "-r", "probe.prmtop",
    "-p", "dock.prm",
    "-n", "50"
]

# Run the docking command
print(f"Running docking with command: {' '.join(params)}")
result = subprocess.run(params, capture_output=True, text=True, env=env)

# Print the output
print(result.stdout)

Running docking with command: rbdock -i /home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/examples/Fake_Crystal.sdf -o docking_results -r probe.prmtop -p dock.prm -n 50
***********************************************
The RxDock molecular docking program is licensed under GNU LGPL version 3.
RxDock is maintained by Vedran Miletić, Patrik Nikolić, and Luka Vretenar.
Visit https://www.rxdock.org/ for more information.
Executable:	rbdock/0.1.0
Library:	librxdock.so/0.1.0
RBT_ROOT:	/home/hitesit/Software/rxdock/rxdock_installation
RBT_HOME:	/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/Prototypes
Current dir:	/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/Prototypes
Date:		Fri May 16 17:58:20 2025
***********************************************

Command line args:
 -i /home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/examples/Fake_Crystal.sdf
 -r probe.prmtop
 -p dock.prm
 -o docking_results
 -n 50
Reading polar hydrogens on